In [1]:
from tqdm import tqdm
import time
import openpyxl
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
## 크롤링 모듈설치

#코랩에서 셀레니움 사용하기 위한 코드
!pip install selenium
#!apt-get update
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import urllib.request 
from urllib.request import urlopen
from urllib.parse import urljoin
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

In [3]:
# 크롬드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko") # 우회를 위해 헤더 설정

driver = webdriver.Chrome('chromedriver', options=options)

In [36]:
df=pd.read_excel('네이버url.xlsx') #결측치 처리 완료
df

,상호명,naver_map_url
0,고집돌우럭 중문점,https://pcmap.place.naver.com/restaurant/76619...
1,고집돌우럭 제주공항점,https://pcmap.place.naver.com/restaurant/38265...
2,고집돌우럭 함덕점,https://pcmap.place.naver.com/restaurant/11773...
3,제주김만복 본점,https://pcmap.place.naver.com/restaurant/17739...
4,문개항아리 조천본점,https://pcmap.place.naver.com/restaurant/37197...
5,델문도,https://pcmap.place.naver.com/restaurant/36859...
6,장인의집,https://pcmap.place.naver.com/restaurant/17271...


In [31]:
#크롬 드라이버 설정
driver = webdriver.Chrome(executable_path=r'C:/Users/silvi/Desktop/jupyter/chromedriver')

# 수집할 정보들
rating_list = [] # 평점
user_review_id = {} # 유저의 id 
review_json = {} # 리뷰
image_json = {} # 이미지

for i in range(6, 7):
    
    print('======================================================')
    print(str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    thisurl = df['naver_map_url'][i]
    time.sleep(2)
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            time.sleep(1)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(3)
            driver.find_element_by_css_selector('#app-root > div > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div.place_section.cXO6M > div._2kAri > a').click()
            time.sleep(3)
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)
            
        except NoSuchElementException:
            print('-더보기 버튼 모두 클릭 완료-')
            break

    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['상호명'][i]
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {}
    review_json[restaurant_name] = {}
    image_json[restaurant_name] = {}
    
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        ul_review = soup.find('ul', attrs = {'class':'_1jVSG'})
        one_review = ul_review.find_all('li', attrs = {'class':'_3FaRE'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))

        for i in range(len(one_review)):

            # user id
            href = one_review[i].find_all(name='a', attrs={"class":"_6jx3q"})
            for z in href:
                user_url=z.get('href')
                print('user_url =' + user_url)

            # user code
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)

            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0]
            review_id = str(res_code)+"_"+user_code
            print('review_id = '+review_id)

            # rating, 별점
            try:
                rating = one_review[i].find('div', attrs = {'class':'_3-LAD'}).text
                #rating = pre_rating.find('span', attrs = {'class':'_1fvo3 Sv1wj'}).text
                print('rating = '+ rating)
            except NoSuchElementException:
                rating=""
                print('rating = NaN')

            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
                
            except: # 리뷰가 없다면
                review_content = ""
            print('리뷰 내용 : '+review_content)

            #리뷰정보
            # user_review_id
            user_review_id[restaurant_name][user_code] = review_id

            # review_json
            review_json[restaurant_name][review_id] = review_content

            # image_json
            #image_json[restaurant_name][review_id] = img_url

            # rating_df_list
            naver_review = user_code, restaurant_name, rating, review_content
            rating_list.append(naver_review)

    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            #리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review

            # rating_df_list
            naver_review = user_code, restaurant_name, none_review, none_reivew
            rating_list.append(naver_review)
            
            
    print('\n')

5번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 델문도
식당 구분 : 카페,디저트
----------------------------------------------
리뷰 총 개수 : 4790
user_url =https://m.place.naver.com/my/5d359284586bd075a2cbaa2c/review?v=2
user_code = 5d359284586bd075a2cbaa2c
review_id = 36859834_5d359284586bd075a2cbaa2c
rating = 최근 방문일4.4.월2022년 4월 4일 월요일1번째 방문영수증
리뷰 내용 : 제주 바다를 품고있는 로스터리 카페.. 한라봉에이드 마시러 가봤고요 😄 내부 사진 한컷은 손님들 안계실때 몰래 🤭 낮에는 가족, 밤에는 연인을 위한 😆
user_url =https://m.place.naver.com/my/5efdcd8c8b790375a186abbf/review?v=2
user_code = 5efdcd8c8b790375a186abbf
review_id = 36859834_5efdcd8c8b790375a186abbf
rating = 최근 방문일4.5.화2022년 4월 5일 화요일1번째 방문영수증
리뷰 내용 : 빵도 맛있고 아이스크림도 최고!
user_url =https://m.place.naver.com/my/5e466b208f87a842bc0c621a/review?v=2
user_code = 5e466b208f87a842bc0c621a
review_id = 36859834_5e466b208f87a842bc0c621a
rating = 최근 방문일4.13.수2022년 4월 13일 수요일1번째 방문영수증
리뷰 내용 : 뷰좋고 쉬어가기 좋은거 같아요~
user_url =https://m.place.naver.com/my/5b9bff398776d6234fd3a69b/review?v=2
user_code = 5b9bff398776d6234fd3a69b
review_id = 368598

리뷰 내용 : 
user_url =https://m.place.naver.com/my/5ee761b08f87a842bc459df5/review?v=2
user_code = 5ee761b08f87a842bc459df5
review_id = 36859834_5ee761b08f87a842bc459df5
rating = 최근 방문일2.27.일2022년 2월 27일 일요일1번째 방문영수증
리뷰 내용 : 함덕은 역시 델문도 👍
user_url =https://m.place.naver.com/my/5e9d6c678f87a842bcba291e/review?v=2
user_code = 5e9d6c678f87a842bcba291e
review_id = 36859834_5e9d6c678f87a842bcba291e
rating = 최근 방문일3.4.금2022년 3월 4일 금요일1번째 방문영수증
리뷰 내용 : 가격은 비싼편이지만, 바다가 보이는 뷰는 정말 😋
user_url =https://m.place.naver.com/my/5c209050a64712dff2630ced/review?v=2
user_code = 5c209050a64712dff2630ced
review_id = 36859834_5c209050a64712dff2630ced
rating = 최근 방문일4.2.토2022년 4월 2일 토요일1번째 방문영수증
리뷰 내용 : 너무 이쁜 카페~
user_url =https://m.place.naver.com/my/5ecd8aba8f87a842bcbc200f/review?v=2
user_code = 5ecd8aba8f87a842bcbc200f
review_id = 36859834_5ecd8aba8f87a842bcbc200f
rating = 최근 방문일2.18.금2022년 2월 18일 금요일1번째 방문영수증
리뷰 내용 : -화려한 그릇 속 남루한 내용물- 카페 위치 및 뷰는 완벽합니다. 제주에서 관광객이 기대하는 이상적인 모습이죠 다만, 카페 안으로 들어가서 바리스타
의 응대부터 실망

review_id = 36859834_5f046bd70b8c7a1cdb6f749c
rating = 최근 방문일3.14.월2022년 3월 14일 월요일1번째 방문영수증
리뷰 내용 : 
user_url =https://m.place.naver.com/my/5d9e87a28f87a842bcd142bd/review?v=2
user_code = 5d9e87a28f87a842bcd142bd
review_id = 36859834_5d9e87a28f87a842bcd142bd
rating = 별점5최근 방문일21.10.13.수2021년 10월 13일 수요일2번째 방문영수증
리뷰 내용 : 베이커리도 맛있고 커피도 맛있어요 말차라떼 최고 !
함덕해수욕장이 바로 앞이라서 좋아요
user_url =https://m.place.naver.com/my/5db2e9ff8f87a842bc432e3e/review?v=2
user_code = 5db2e9ff8f87a842bc432e3e
review_id = 36859834_5db2e9ff8f87a842bc432e3e
rating = 최근 방문일21.12.19.일2021년 12월 19일 일요일1번째 방문영수증
리뷰 내용 : 사람이 너~~~무 많아요ㅠㅠㅠ
빵 종류도 많아요!!
user_url =https://m.place.naver.com/my/5e00e3c28f87a842bc36d58b/review?v=2
user_code = 5e00e3c28f87a842bc36d58b
review_id = 36859834_5e00e3c28f87a842bc36d58b
rating = 최근 방문일3.13.일2022년 3월 13일 일요일1번째 방문영수증
리뷰 내용 : 
user_url =https://m.place.naver.com/my/5dda5f1c9ec8258e4aa743cd/review?v=2
user_code = 5dda5f1c9ec8258e4aa743cd
review_id = 36859834_5dda5f1c9ec8258e4aa743cd
rating = 

커피맛집은 아닌듯
user_url =https://m.place.naver.com/my/5ddc8eb39ec8258e4a90d62a/review?v=2
user_code = 5ddc8eb39ec8258e4a90d62a
review_id = 36859834_5ddc8eb39ec8258e4a90d62a
rating = 최근 방문일21.12.15.수2021년 12월 15일 수요일1번째 방문영수증
리뷰 내용 : 뷰맛집
커피맛집.
user_url =https://m.place.naver.com/my/5efde4db15c4bcd43058687f/review?v=2
user_code = 5efde4db15c4bcd43058687f
review_id = 36859834_5efde4db15c4bcd43058687f
rating = 최근 방문일4.7.목2022년 4월 7일 목요일1번째 방문영수증
리뷰 내용 : 좋아요
user_url =https://m.place.naver.com/my/5da0a20f8f87a842bcaff129/review?v=2
user_code = 5da0a20f8f87a842bcaff129
review_id = 36859834_5da0a20f8f87a842bcaff129
rating = 최근 방문일21.12.10.금2021년 12월 10일 금요일2번째 방문영수증
리뷰 내용 : 바다뷰를 보면서 마시는 커피~
분위기도 넘 좋아요
user_url =https://m.place.naver.com/my/5d1185122168f058a19c2463/review?v=2
user_code = 5d1185122168f058a19c2463
review_id = 36859834_5d1185122168f058a19c2463
rating = 최근 방문일2.24.목2022년 2월 24일 목요일1번째 방문영수증
리뷰 내용 : 
user_url =https://m.place.naver.com/my/5e579a8c8f87a842bc2093a7/review?v=2
user_code = 

rating = 최근 방문일21.12.13.월2021년 12월 13일 월요일1번째 방문영수증
리뷰 내용 : 북적북적 서비스개판 융통성제로 예전같지않네요 다른뷰좋은 카페가세요 다시 안올것.
user_url =https://m.place.naver.com/my/5f0590aa2f90b7de35380eef/review?v=2
user_code = 5f0590aa2f90b7de35380eef
review_id = 36859834_5f0590aa2f90b7de35380eef
rating = 최근 방문일1.27.목2022년 1월 27일 목요일1번째 방문영수증
리뷰 내용 : 
user_url =https://m.place.naver.com/my/616e0d1fe771331b06b53859/review?v=2
user_code = 616e0d1fe771331b06b53859
review_id = 36859834_616e0d1fe771331b06b53859
rating = 최근 방문일3.16.수2022년 3월 16일 수요일2번째 방문영수증
리뷰 내용 : 굿
user_url =https://m.place.naver.com/my/5f26120d9ec8258e4a85f01e/review?v=2
user_code = 5f26120d9ec8258e4a85f01e
review_id = 36859834_5f26120d9ec8258e4a85f01e
rating = 최근 방문일3.16.수2022년 3월 16일 수요일1번째 방문영수증
리뷰 내용 : 굿
user_url =https://m.place.naver.com/my/5c1f63f64680feb9f1bbb2a8/review?v=2
user_code = 5c1f63f64680feb9f1bbb2a8
review_id = 36859834_5c1f63f64680feb9f1bbb2a8
rating = 최근 방문일3.16.수2022년 3월 16일 수요일1번째 방문영수증
리뷰 내용 : 뷰맛집
user_url =https://m.place.naver.co

...

review_id = 36859834_5b9e624841d458e68d0117b4
rating = 별점4최근 방문일20.6.22.월2020년 6월 22일 월요일1번째 방문영수증
리뷰 내용 : 제주도 여행 왔다면 한번은 꼭 가볼만한 뷰 맛집.
빵도 맛있고 커피도 나쁘지 않았습니다
user_url =https://m.place.naver.com/my/5dff6a588f87a842bccf09a2/review?v=2
user_code = 5dff6a588f87a842bccf09a2
review_id = 36859834_5dff6a588f87a842bccf09a2
rating = 별점0.5최근 방문일20.5.30.토2020년 5월 30일 토요일1번째 방문영수증
리뷰 내용 : 기분좋게 여행 마무리 하려고 뷰좋은 카페 왔는데 서비스 완전 별로ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ빨대 뒤에 있다고 해서 봤더니 잘 안보여서 어디쪽에 있냐고 조심스럽게 물어봤는데 저기요!!!! 하면서 짜증내구 계산하는 분도 너무 불친절. 바쁜건 알지만 좀 그래요. 지인들한테 추천하고 싶지않은 정도. 뷰 보고 오시는 분들은 괜찮을듯!
user_url =https://m.place.naver.com/my/5eca54f48f87a842bc8bb825/review?v=2
user_code = 5eca54f48f87a842bc8bb825
review_id = 36859834_5eca54f48f87a842bc8bb825
rating = 별점5최근 방문일21.3.10.수2021년 3월 10일 수요일1번째 방문영수증
리뷰 내용 : 굿
user_url =https://m.place.naver.com/my/5f061c87c6c911a490bd5b04/review?v=2
user_code = 5f061c87c6c911a490bd5b04
review_id = 36859834_5f061c87c6c911a490bd5b04
rating = 별점5최근 방문일20.11.4.수2020년 11월 4일 수요일1번째 방문영수증
리뷰 내용 : 좋아요
u

user_url =https://m.place.naver.com/my/5b701d09930fac61e54386d6/review?v=2
user_code = 5b701d09930fac61e54386d6
review_id = 36859834_5b701d09930fac61e54386d6
rating = 별점4.5최근 방문일20.5.8.금2020년 5월 8일 금요일1번째 방문영수증
리뷰 내용 : 바다뷰가 좋아 한참을 앉아 있었다. 아메리카노도 좋았다.
user_url =https://m.place.naver.com/my/5b9a7c9ae1d53b17f7c46f96/review?v=2
user_code = 5b9a7c9ae1d53b17f7c46f96
review_id = 36859834_5b9a7c9ae1d53b17f7c46f96
rating = 별점5최근 방문일20.7.19.일2020년 7월 19일 일요일1번째 방문영수증
리뷰 내용 : 바다 바로 앞이라 좋아요!
user_url =https://m.place.naver.com/my/5c78bdf27c997603c104a565/review?v=2
user_code = 5c78bdf27c997603c104a565
review_id = 36859834_5c78bdf27c997603c104a565
rating = 별점5최근 방문일20.7.8.수2020년 7월 8일 수요일1번째 방문영수증
리뷰 내용 : 썬비치 자리 좋아요~
user_url =https://m.place.naver.com/my/5b9786523994012c075afde3/review?v=2
user_code = 5b9786523994012c075afde3
review_id = 36859834_5b9786523994012c075afde3
rating = 별점4최근 방문일20.12.14.월2020년 12월 14일 월요일1번째 방문영수증
리뷰 내용 : 좋아요 바다와 커피
user_url =https://m.place.naver.com/my/5e5b271a8f87a84

rating = 별점4최근 방문일20.6.27.토2020년 6월 27일 토요일1번째 방문영수증
리뷰 내용 : 바다뷰가 좋아요 ㅎㅎ
user_url =https://m.place.naver.com/my/5ef02f0e8f87a842bcc379d8/review?v=2
user_code = 5ef02f0e8f87a842bcc379d8
review_id = 36859834_5ef02f0e8f87a842bcc379d8
rating = 별점5최근 방문일20.8.7.금2020년 8월 7일 금요일4번째 방문영수증
리뷰 내용 : 좋아요
user_url =https://m.place.naver.com/my/5b84e79f5db412cab3ef52f6/review?v=2
user_code = 5b84e79f5db412cab3ef52f6
review_id = 36859834_5b84e79f5db412cab3ef52f6
rating = 별점5최근 방문일19.11.27.수2019년 11월 27일 수요일1번째 방문영수증
리뷰 내용 : 장점 : 뷰가 너무 좋아요. 커피와 빵이 맛있어요. 특히 마농(마늘빵) 꼭 드세요. 커피는 부드럽고 향긋한 맛이였어요 직접 로스팅도 해서 더더욱 향이 좋
았던듯.. 델문도 주차장아니여도 주변에 주차할곳이 많아요 단점 : 가격이비싸요(이건 제주도가 그런듯)
user_url =https://m.place.naver.com/my/5d3094b3f3622173757309df/review?v=2
user_code = 5d3094b3f3622173757309df
review_id = 36859834_5d3094b3f3622173757309df
rating = 별점5최근 방문일20.8.29.토2020년 8월 29일 토요일1번째 방문영수증
리뷰 내용 : 😊조하여
user_url =https://m.place.naver.com/my/5e74e5838f87a842bc60ff20/review?v=2
user_code = 5e74e5838f87a842bc60ff20
review

rating = 별점5최근 방문일20.1.7.화2020년 1월 7일 화요일1번째 방문영수증
리뷰 내용 : 일찍 가면 사람 없어서 사진찍기 좋아요 빵 맛있어요
user_url =https://m.place.naver.com/my/5f5701126c3d7f73349115e2/review?v=2
user_code = 5f5701126c3d7f73349115e2
review_id = 36859834_5f5701126c3d7f73349115e2
rating = 별점5최근 방문일20.11.18.수2020년 11월 18일 수요일1번째 방문영수증
리뷰 내용 : 바다 위에 떠있는듯한  카페
넘 이뻐요
user_url =https://m.place.naver.com/my/5dddeb9c9ec8258e4a7dc081/review?v=2
user_code = 5dddeb9c9ec8258e4a7dc081
review_id = 36859834_5dddeb9c9ec8258e4a7dc081
rating = 별점3최근 방문일20.7.15.수2020년 7월 15일 수요일1번째 방문영수증
리뷰 내용 : 좋아요~~~~~~~
user_url =https://m.place.naver.com/my/5cf1cdf18f87a842bc4b1534/review?v=2
user_code = 5cf1cdf18f87a842bc4b1534
review_id = 36859834_5cf1cdf18f87a842bc4b1534
rating = 별점5최근 방문일20.1.14.화2020년 1월 14일 화요일1번째 방문영수증
리뷰 내용 : 함덕해변이 열일하는 카페. 가격이 사악하지만 가치있는곳
user_url =https://m.place.naver.com/my/5e8056898f87a842bc0f35f7/review?v=2
user_code = 5e8056898f87a842bc0f35f7
review_id = 36859834_5e8056898f87a842bc0f35f7
rating = 별점5최근 방문일20.4.8.수2020년

rating = 별점3최근 방문일20.1.29.수2020년 1월 29일 수요일1번째 방문영수증
리뷰 내용 : 
user_url =https://m.place.naver.com/my/5efdf95ab1f7027e8a6cb510/review?v=2
user_code = 5efdf95ab1f7027e8a6cb510
review_id = 36859834_5efdf95ab1f7027e8a6cb510
rating = 별점4최근 방문일20.11.2.월2020년 11월 2일 월요일1번째 방문영수증
리뷰 내용 : 경치 좋아요
user_url =https://m.place.naver.com/my/5c12f463d88b7e73d0afc783/review?v=2
user_code = 5c12f463d88b7e73d0afc783
review_id = 36859834_5c12f463d88b7e73d0afc783
rating = 별점4최근 방문일20.1.13.월2020년 1월 13일 월요일1번째 방문영수증
리뷰 내용 : 함덕해수욕장에있는베이커리와커피를마실수있는곳~~경치짱
user_url =https://m.place.naver.com/my/5efe3c2d35e32560edaa9978/review?v=2
user_code = 5efe3c2d35e32560edaa9978
review_id = 36859834_5efe3c2d35e32560edaa9978
rating = 별점5최근 방문일21.1.24.일2021년 1월 24일 일요일2번째 방문영수증
리뷰 내용 : 굿
user_url =https://m.place.naver.com/my/5c01d8ed490d395fc882221b/review?v=2
user_code = 5c01d8ed490d395fc882221b
review_id = 36859834_5c01d8ed490d395fc882221b
rating = 별점5최근 방문일20.1.10.금2020년 1월 10일 금요일1번째 방문영수증
리뷰 내용 : 야경이 넘 좋아요~함덕해변 넘 이쁩니다^^


In [32]:
rating_df7 = pd.DataFrame(rating_list)
rating_df7.columns = ['UserID','ItemID','Rating', 'Review']
#rating_df3.to_csv('고집돌우럭함덕점네이버리뷰.csv', encoding='utf-8-sig')
rating_df7

,UserID,ItemID,Rating,Review
0,5d359284586bd075a2cbaa2c,델문도,최근 방문일4.4.월2022년 4월 4일 월요일1번째 방문영수증,제주 바다를 품고있는 로스터리 카페.. 한라봉에이드 마시러 가봤고요 😄 내부 사진 ...
1,5efdcd8c8b790375a186abbf,델문도,최근 방문일4.5.화2022년 4월 5일 화요일1번째 방문영수증,빵도 맛있고 아이스크림도 최고!
2,5e466b208f87a842bc0c621a,델문도,최근 방문일4.13.수2022년 4월 13일 수요일1번째 방문영수증,뷰좋고 쉬어가기 좋은거 같아요~
3,5b9bff398776d6234fd3a69b,델문도,최근 방문일3.29.화2022년 3월 29일 화요일1번째 방문영수증,뷰만 예쁠줄 알았는데 빵이랑 커피 다 맛있어요!\n참새들이 빵 얻어먹으러 많이 옵니...
4,5b9b1e17819d030463b3b110,델문도,최근 방문일1.8.토2022년 1월 8일 토요일1번째 방문영수증,빵맛 무난해요. 괜찮아요. 그냥 맛보기로 한두개 사먹으면 좋겠어요. 유명 관광지의 ...
...,...,...,...,...
4785,6005231a615cd42308cde7c4,델문도,별점3최근 방문일18.5.29.화2018년 5월 29일 화요일1번째 방문영수증,비싸요 전경은 이쁨
4786,5e0771ef8f87a842bcb189a6,델문도,별점4최근 방문일18.5.29.화2018년 5월 29일 화요일2번째 방문영수증,이뻐요
4787,5b8dc49aafd9765938f3c1df,델문도,별점5최근 방문일17.12.20.수2017년 12월 20일 수요일1번째 방문영수증,
4788,5e8757678f87a842bc51594a,델문도,별점3.5최근 방문일17.9.20.수2017년 9월 20일 수요일1번째 방문영수증,뷰가 예뻐요


In [34]:
rating_df7['Rating']=[z.split('최근')[0]for z in rating_df7['Rating']]
rating_df7['Rating']=[z.replace('별점', ' ') for z in rating_df7['Rating']]
rating_df7

,UserID,ItemID,Rating,Review
0,5d359284586bd075a2cbaa2c,델문도,,제주 바다를 품고있는 로스터리 카페.. 한라봉에이드 마시러 가봤고요 😄 내부 사진 ...
1,5efdcd8c8b790375a186abbf,델문도,,빵도 맛있고 아이스크림도 최고!
2,5e466b208f87a842bc0c621a,델문도,,뷰좋고 쉬어가기 좋은거 같아요~
3,5b9bff398776d6234fd3a69b,델문도,,뷰만 예쁠줄 알았는데 빵이랑 커피 다 맛있어요!\n참새들이 빵 얻어먹으러 많이 옵니...
4,5b9b1e17819d030463b3b110,델문도,,빵맛 무난해요. 괜찮아요. 그냥 맛보기로 한두개 사먹으면 좋겠어요. 유명 관광지의 ...
...,...,...,...,...
4785,6005231a615cd42308cde7c4,델문도,3,비싸요 전경은 이쁨
4786,5e0771ef8f87a842bcb189a6,델문도,4,이뻐요
4787,5b8dc49aafd9765938f3c1df,델문도,5,
4788,5e8757678f87a842bc51594a,델문도,3.5,뷰가 예뻐요


In [35]:
rating_df7.to_csv('장인의집네이버리뷰.csv', encoding='utf-8-sig')